In [249]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [250]:
df_f=pd.read_csv(r'TSC_sales_forecast.csv')
df_e=pd.read_csv(r'Coeffs1.csv')
df_a=pd.read_csv(r'TSC_Sales_Data.csv')

In [251]:
df_e.columns

Index(['SCRUB_ITEM', 'UNIT_PRICE_coeff', 'year_coeff', 'sin365_coeff',
       'cos365_coeff', 'month_coeff', 'week_coeff', 'UNIT_PRICE*month',
       'UNIT_PRICE*year', 'month*week', 'month*year', 'month*year*week',
       'month*year*week*UNIT_PRICE', 'constant'],
      dtype='object')

In [285]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

standardized_X = scaler.fit_transform(df_e[['UNIT_PRICE_coeff', 'year_coeff', 'sin365_coeff',
       'cos365_coeff', 'month_coeff', 'week_coeff', 'UNIT_PRICE*month',
       'UNIT_PRICE*year', 'month*week', 'month*year', 'month*year*week',
       'month*year*week*UNIT_PRICE', 'constant']])
col = ['UNIT_PRICE_coeff', 'year_coeff', 'sin365_coeff',
       'cos365_coeff', 'month_coeff', 'week_coeff', 'UNIT_PRICE*month',
       'UNIT_PRICE*year', 'month*week', 'month*year', 'month*year*week',
       'month*year*week*UNIT_PRICE', 'constant']
df_e_1 = pd.DataFrame(standardized_X, columns =col)
df_e_1[col] = df_e_1[col]/10
df_e_1['SCRUB_ITEM'] = df_e['SCRUB_ITEM']


In [286]:
df_e_1.head()

,UNIT_PRICE_coeff,year_coeff,sin365_coeff,cos365_coeff,month_coeff,week_coeff,UNIT_PRICE*month,UNIT_PRICE*year,month*week,month*year,month*year*week,month*year*week*UNIT_PRICE,constant,SCRUB_ITEM
0,0.011911,0.023248,0.004107,0.009448,0.004107,-0.008769,0.000206,-0.011910,0.003082,0.007541,-0.006793,0.007669,-0.023158,24
1,0.009869,0.009456,-0.005970,-0.041692,-0.005970,-0.020653,0.018935,-0.009870,0.003120,-0.004423,-0.006582,0.002574,-0.009349,47
2,0.010423,0.014842,-0.016151,0.047377,-0.016151,-0.006208,0.015439,-0.010423,0.010399,0.019200,-0.013362,0.003913,-0.014741,7
3,0.009003,0.005323,-0.036796,0.072503,-0.036796,-0.001668,0.006236,-0.009004,0.040302,0.097939,-0.040877,0.007660,-0.005224,31
4,0.010303,0.015972,0.000938,0.005953,0.000938,-0.011369,0.015375,-0.010304,-0.004320,-0.024260,0.000118,0.003994,-0.015869,71


In [287]:
df=pd.merge(df_f,df_e_1,how='left',on=['SCRUB_ITEM'])

df_price=df_a.groupby(['SCRUB_ITEM'])['UNIT_PRICE'].max().reset_index()
df=pd.merge(df,df_price,how='left',on=['SCRUB_ITEM'])
df['key']=df['SCRUB_ITEM'].astype(str)+'_'+df['weekday'].astype(str)+'_'+df['year'].astype(str)
df_price=df_a.groupby(['SCRUB_ITEM'])['UNIT_COST'].max().reset_index()
df=pd.merge(df,df_price,how='left',on=['SCRUB_ITEM'])
df['month']=df['weekday']//4.2
df['month']=df['month'].astype(int)
df['UNIT_QTY']=df['UNIT_QTY'].abs()

In [288]:
df.columns

Index(['SCRUB_ITEM', 'weekday', 'year', 'UNIT_QTY', 'sin365', 'cos365',
       'UNIT_PRICE_coeff', 'year_coeff', 'sin365_coeff', 'cos365_coeff',
       'month_coeff', 'week_coeff', 'UNIT_PRICE*month', 'UNIT_PRICE*year',
       'month*week', 'month*year', 'month*year*week',
       'month*year*week*UNIT_PRICE', 'constant', 'UNIT_PRICE', 'key',
       'UNIT_COST', 'month'],
      dtype='object')

In [289]:
df_1 = df_f[df_f.year == 2021]
df_1 = df_f[df_f.weekday>31]
df_1.SCRUB_ITEM.nunique()

96

In [290]:
df_rp=pd.read_csv(r'Competitors_Price_Changes.csv')

In [291]:
df_rp.SCRUB_ITEM.nunique()

97

In [292]:
df_rp.head()

,SCAN_DATE_TIME,SCRUB_ITEM,SCRUB_COMPETITOR,TSC_PRICE,COMPETITOR_PRICE,CPI_COMPETITIVE_PRICE_INDEX,COMPETITOR_SHIPPING,COMPETITOR_REG_PRICE
0,2020-08-06 04:20:00.000,1,1,199.99,205.58,102.7951,0.0,205.58
1,2020-08-06 04:21:00.000,1,2,199.99,205.58,102.7951,0.0,205.58
2,2020-08-07 04:24:00.000,1,1,199.99,205.58,102.7951,0.0,205.58
3,2020-08-07 04:24:00.000,1,2,199.99,205.58,102.7951,0.0,205.58
4,2020-08-08 04:22:00.000,1,1,199.99,205.58,102.7951,0.0,205.58


In [293]:
df_price=df_rp.groupby(['SCRUB_ITEM'])['TSC_PRICE'].max().reset_index()
df=pd.merge(df,df_price,how='left',on=['SCRUB_ITEM'])

In [294]:
df.head()

,SCRUB_ITEM,weekday,year,UNIT_QTY,sin365,cos365,UNIT_PRICE_coeff,year_coeff,sin365_coeff,cos365_coeff,...,month*week,month*year,month*year*week,month*year*week*UNIT_PRICE,constant,UNIT_PRICE,key,UNIT_COST,month,TSC_PRICE
0,2,1,2018,3.0,7.485107e-01,6.631227e-01,-0.979389,-0.958921,-0.293941,-0.385267,...,0.024594,0.048708,-0.029903,0.602171,0.958948,14.99,2_1_2018,10.99,0,14.99
1,2,52,2018,13.0,-1.714506e-15,1.000000e+00,-0.979389,-0.958921,-0.293941,-0.385267,...,0.024594,0.048708,-0.029903,0.602171,0.958948,14.99,2_52_2018,10.99,12,14.99
2,2,1,2019,21.0,6.631227e-01,7.485107e-01,-0.979389,-0.958921,-0.293941,-0.385267,...,0.024594,0.048708,-0.029903,0.602171,0.958948,14.99,2_1_2019,10.99,0,14.99
3,2,2,2019,11.0,1.000000e+00,-1.608123e-16,-0.979389,-0.958921,-0.293941,-0.385267,...,0.024594,0.048708,-0.029903,0.602171,0.958948,14.99,2_2_2019,10.99,0,14.99
4,2,3,2019,32.0,6.631227e-01,-7.485107e-01,-0.979389,-0.958921,-0.293941,-0.385267,...,0.024594,0.048708,-0.029903,0.602171,0.958948,14.99,2_3_2019,10.99,0,14.99


In [296]:
df.UNIT_PRICE_coeff.min()
df['UNIT_PRICE_coeff'] = df['UNIT_PRICE_coeff']/10

In [297]:
df_1 = df[df.year == 2021]
df_1 = df_1[df_1.weekday>31]

In [ ]:
s=[]
p=[]
k=[]
#count_1 = 0

for i in df_1['key'].unique():
    #print(count_1)
    #count_1 = count_1+1
    df_dummy=df[df['key']==i]
    df_dummy.index=range(0,len(df_dummy))
    df_dummy=df_dummy.rename(columns={'weekday':'week'})
    CP=df_dummy['UNIT_COST'][0]
    def objective(x):
        return ((x[0]*CP) - (x[0]*x[1]))
    def constraint1(x):
        a=  -x[0] + df_dummy['UNIT_PRICE_coeff']*x[1] + df_dummy['sin365_coeff']*df_dummy['sin365'] + df_dummy['cos365_coeff']*df_dummy['cos365']+df_dummy['year_coeff']* df_dummy['year']+df_dummy['month_coeff']*df_dummy['month']+df_dummy['week_coeff']*df_dummy['week']+df_dummy['UNIT_PRICE*month']*x[1]*df_dummy['month']+df_dummy['UNIT_PRICE*year']*x[1]*df_dummy['year']+df_dummy['month*week']*df_dummy['month']*df_dummy['week']+df_dummy['month*year']*df_dummy['month']*df_dummy['year']+df_dummy['month*year*week']*df_dummy['month']*df_dummy['week']*df_dummy['year']+df_dummy['month*year*week*UNIT_PRICE']*df_dummy['month']*df_dummy['year']*x[1]*df_dummy['week']+df_dummy['constant']
        return a[0]   
    n = 2
    x0 = np.zeros(2)
    x0[0] = df_dummy['UNIT_QTY'][0] # Forecast value
    x0[1] = df_dummy['UNIT_COST'][0] #Current price
    
    
    b1 = (df_dummy['UNIT_QTY'][0],1.01*df_dummy['UNIT_QTY'][0]) #sales bin
    b2 = (df_dummy['UNIT_COST'][0],df_dummy['TSC_PRICE'][0]) #price bin - cp, max_retail_price
    bnds = (b1,b2)
    
    con1 = {'type': 'eq', 'fun': constraint1}
    
    cons = ([con1])
    solution = minimize(objective,x0,method='SLSQP',bounds=bnds,constraints=cons)
    x = solution.x
    s.append(x[0])
    p.append(x[1])
    k.append(i)
df_price_rec=pd.DataFrame()
df_price_rec['key']=k
df_price_rec['price']=p
df_price_rec['sales']=s

In [ ]:
df_price_rec.to_csv('Optimise_Price.csv',index = False)